Objective: evaluate latency of packets sent by GUs to a Remote reachable through Wi-Fi - Relay - LTE - Internet Backbone

**TODO**
* Generalize block of code to extract tx'd pkts for all drones in the wifi networks


In [2]:
import re

import pandas as pd
import plotly.express as px


In [3]:
base_results_dir = '../results/'
scenario_name = 'paper_3-wifi'
scenario_dir = f'{scenario_name}-2022-02-28.17-07-31'
wifi_tx_trace_filename = 'wifi-phy-0-drones-host-4-0.log'

rex = re.compile(r't (?P<time>[0-9\.+]+).+HeMcs(?P<mcs>[0-9]+).+200\.0\.0\.1.+ns3::SeqTsHeader \(\(seq=(?P<sn>[0-9]+).+')
pkts = []

with open(f'{base_results_dir}/{scenario_dir}/{wifi_tx_trace_filename}', 'r') as f:
  for l in f:
    r = rex.match(l)

    if r is not None:
      ## DEBUG
      # print(r[0])
      # print(r.groupdict())
      # break
      ##
      pkts.append({k: float(v) for k,v in r.groupdict().items()})  

tx = pd.DataFrame(pkts)
tx.to_csv(f'{base_results_dir}/{scenario_dir}/wifi-phy-0-host-4-0.csv')

In [4]:
# "7.0.0.2:4": "10.1.0.2"
internet_rx_trace_filename = 'internet-29-1.tr'

rex = re.compile(r'r (?P<time>[0-9\.+]+).+7\.0\.0\.2.+4 > 1337.+ns3::SeqTsHeader \(\(seq=(?P<sn>[0-9]+).+')
pkts = []

with open(f'{base_results_dir}/{scenario_dir}/{internet_rx_trace_filename}', 'r') as f:
  for l in f:
    r = rex.match(l)

    if r is not None:
      ## DEBUG
      # print(r[0])
      # print(r.groupdict())
      # break
      ##
      pkts.append({k: float(v) for k,v in r.groupdict().items()})

rx = pd.DataFrame(pkts)
rx.to_csv(f'{base_results_dir}/{scenario_dir}/wifi-phy-0-host-4-0.internet.csv')

In [5]:
txgp = tx.groupby(['sn']).first()
latency = pd.merge(txgp, rx, on='sn', how='outer')

latency['time_x'] = pd.to_numeric(latency['time_x'])
latency['time_y'] = pd.to_numeric(latency['time_y'])
latency['10.1.0.1'] = latency.time_y - latency.time_x

In [9]:
fig = px.box(latency['10.1.0.1'])
fig.show()